In [1]:
import pandas as pd
import datetime
import requests
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup # Alert: BS4 no longer work for live price data as Yahoo is updating the class name in time


In [2]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service


In [3]:
PATH = "C:\Program Files\chromedriver.exe"
service = Service(executable_path=PATH)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # not show chrome window
driver = webdriver.Chrome(service=service, options=chrome_options)


In [4]:
def xpath_element(xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)

    except NoSuchElementException:
        element = []

    return element

def real_time_price(stock_code):
    url = "https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch".format(stock_code,stock_code)
    driver.get(url)

    # First getting the Price and Price change
    xpath = '//*[@id="quote-header-info"]/div[3]/div[1]/div[1]'
    stock_price_info = xpath_element(xpath=xpath)
    

    if(stock_price_info != []):
        stock_price_temp = stock_price_info.text.split()[0]
        if stock_price_temp.find('+') != -1:
            price = stock_price_temp.split('+')[0]
            try:
                change = '+' + stock_price_temp.split('+')[1] + ' ' + stock_price_info.text.split()[1]
            except IndexError:
                change = []

        elif stock_price_temp.find('-') != -1:
            price = stock_price_temp.split('-')[0]
            try:
                change = '-' + stock_price_temp.split('-')[1] + ' ' + stock_price_info.text.split()[1]
            except IndexError:
                change = []

        elif stock_price_temp.find('0.00') != -1:
            price = stock_price_temp[0:-4]
            try:
                change = '+' + stock_price_temp[-4:] + ' ' + stock_price_info.text.split()[1]
            except IndexError:
                change = []

        else:
            print(" prev searching failed")
            price, change = [], []
    
    else:
        price, change = [], []
    print("price is",price)
    print(f"change is {change}")

        
    # Second getting the Volume 
    xpath = '//*[@id="quote-summary"]/div[1]'
    left_chart_info = xpath_element(xpath=xpath)
    if(left_chart_info != []):
        # approach 1: Default off. Uncomment to activate
        #volume = left_chart_info.text.split()[-4]

        # approach 2: Safer!
        for i, text in enumerate(left_chart_info.text.split()):
            if text =='Volume':
                volume = left_chart_info.text.split()[i+1]
                break
            else:
                volume = []
        print("volume is ",volume)
    
    else:
        volume = []

        
    #Now getting the 1 year est
    xpath = '//*[@id="quote-summary"]/div[2]'
    right_chart_info = xpath_element(xpath=xpath)
    if(right_chart_info != []):
        # approach 1: Default off. Uncomment to activate
        #volume = left_chart_info.text.split()[-1]

        # approach 2: Safer!
        for i, text in enumerate(right_chart_info.text.split()):
            if text =='Est':
                if right_chart_info.text.split()[i+1] != 'N/A':

                    one_year_target = right_chart_info.text.split()[i+1]
                else:
                    one_year_target = []
                break
            else:
                one_year_target = []
        print(one_year_target)
    
    else:
        one_year_target = []
    
    print("one year tar", one_year_target)

    latest_pattern = []
    #price, change, volume, latest_pattern, one_year_target = [], [], [], [], []

    return price, change, volume, latest_pattern, one_year_target


In [5]:
Stock = ['IONQ', 'MSFT', "NVDA",]

In [6]:

while(True):
    info = []
    # Shift to US EST time 
    time_stamp = datetime.datetime.now() + datetime.timedelta(hours=1) # 1 hour faster than time in Nashville
    time_stamp = time_stamp.strftime("%Y-%m-%d %H:%M:%S")

    for stock_code in Stock:
        stock_price, change, volume,latest_pattern, one_year_target= real_time_price(stock_code)
        info.append(stock_code)
        info.append(stock_price)
        info.extend([change])
        info.extend([volume])
        info.extend([one_year_target])
    
    col = [time_stamp]
    col.extend(info)
    df = pd.DataFrame(col)
    df = df.T
    path = "C:\\Users\\User\\Desktop\\QuantPack\\"
    path += str(time_stamp[0:11])+ 'stock_data.csv'

    df.to_csv(path, mode='a', header=False)
    
   
        

price is 11.91
change is -0.24 (-2.01%)
volume is  3,423,012
16.00
one year tar 16.00
price is 371.83
change is +0.96 (+0.26%)
volume is  9,766,440
409.61
one year tar 409.61
price is 477.51
change is -4.17 (-0.87%)
volume is  19,146,930
641.23
one year tar 641.23
price is 11.91
change is -0.24 (-2.01%)
volume is  3,423,158
16.00
one year tar 16.00
price is 371.79
change is +0.92 (+0.25%)
volume is  9,772,235
409.61
one year tar 409.61
price is 477.59
change is -4.09 (-0.85%)
volume is  19,148,600
641.23
one year tar 641.23
price is 11.91
change is -0.24 (-2.01%)
volume is  3,423,158
16.00
one year tar 16.00
price is 371.80
change is +0.93 (+0.25%)
volume is  9,772,951
409.61
one year tar 409.61
price is 477.57
change is -4.11 (-0.85%)
volume is  19,151,397
641.23
one year tar 641.23
price is 11.91
change is -0.24 (-2.01%)
volume is  3,423,274
16.00
one year tar 16.00
price is 371.76
change is +0.89 (+0.24%)
volume is  9,774,130
409.61
one year tar 409.61
price is 477.53
change is -4.1

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome-headless-shell=120.0.6099.130)
Stacktrace:
	GetHandleVerifier [0x00007FF6F38E2142+3514994]
	(No symbol) [0x00007FF6F3500CE2]
	(No symbol) [0x00007FF6F33A76AA]
	(No symbol) [0x00007FF6F3390839]
	(No symbol) [0x00007FF6F33906EB]
	(No symbol) [0x00007FF6F338EE3D]
	(No symbol) [0x00007FF6F338F7CE]
	(No symbol) [0x00007FF6F339D6CD]
	(No symbol) [0x00007FF6F33B4471]
	(No symbol) [0x00007FF6F33B937A]
	(No symbol) [0x00007FF6F338FEC6]
	(No symbol) [0x00007FF6F33B41CF]
	(No symbol) [0x00007FF6F3432824]
	(No symbol) [0x00007FF6F3415D93]
	(No symbol) [0x00007FF6F33E4BDC]
	(No symbol) [0x00007FF6F33E5C64]
	GetHandleVerifier [0x00007FF6F390E16B+3695259]
	GetHandleVerifier [0x00007FF6F3966737+4057191]
	GetHandleVerifier [0x00007FF6F395E4E3+4023827]
	GetHandleVerifier [0x00007FF6F36304F9+689705]
	(No symbol) [0x00007FF6F350C048]
	(No symbol) [0x00007FF6F3508044]
	(No symbol) [0x00007FF6F3508